# Minimalistic CutFEM Example

In [1]:
using Gridap
using LinearAlgebra
using GridapEmbedded
using GridapGmsh
using Parameters

# α(x) = x[1]^2 + x[2]^2
α(x) = 1
function minimalistic_CutFEM(;order, n, u_ex)

    ∇u_ex(x) = ∇(u_ex)(x)
    ∇Δu_ex(x) = ∇(Δ(u_ex))(x)
    f(x) = Δ(Δ(u_ex))(x)+ α(x)⋅u_ex(x)
        
    println("n=",n)
    # Background model
    L = 1.11
    domain = (-L, L, -L, L)
    pmin = Point(-L, -L)
    pmax = Point(L, L)
    partition = (n,n)
    bgmodel = CartesianDiscreteModel(pmin, pmax, partition)

    # Implicit geometry
    R  = 1.0
    geo = disk(R)

    # Cut the background model
    cutgeo = cut(bgmodel, geo)
    cutgeo_facets = cut_facets(bgmodel,geo)

    # Set up interpolation mesh and function spaces
    Ω_act = Triangulation(cutgeo, ACTIVE)

    # Set up interpolation mesh and function spaces
    Ω_act = Triangulation(cutgeo, ACTIVE)

    # Construct function spaces
    V = TestFESpace(Ω_act, ReferenceFE(lagrangian, Float64, order), conformity=:H1)
    U = TrialFESpace(V)

    # Set up integration meshes, measures and normals
    Ω = Triangulation(cutgeo, PHYSICAL)
    Γ = EmbeddedBoundary(cutgeo)
    Λ = SkeletonTriangulation(cutgeo_facets)
    Fg = GhostSkeleton(cutgeo)

    # Set up integration measures
    degree = 2*order
    dΩ   = Measure(Ω, degree)
    dΓ   = Measure(Γ, degree)
    dΛ   = Measure(Λ, degree) # F_int
    dFg  = Measure(Fg, degree)

    # Set up normal vectors
    n_Γ = get_normal_vector(Γ)
    n_Λ = get_normal_vector(Λ)
    n_Fg = get_normal_vector(Fg)

    # Define weak form
    γ = 25*order*( order+1)

    # Ghost penalty parameter
    γg0 = γ
    γg1 = 0.1
    γg2 = 0.1

    # Mesh size
    h = L/n


    function mean_n(u,n)
        return 0.5*( u.plus⋅n.plus + u.minus⋅n.minus )
    end

    function mean_nn(u,n)
        return 0.5*( n.plus⋅ ∇∇(u).plus⋅ n.plus + n.minus ⋅ ∇∇(u).minus ⋅ n.minus )
    end

    function jump_nn(u,n)
        return ( n.plus⋅ ∇∇(u).plus⋅ n.plus - n.minus ⋅ ∇∇(u).minus ⋅ n.minus )
    end
    # Inner facets
    a(u,v) =( ∫( ∇∇(v)⊙∇∇(u) + α⋅(v⊙u) )dΩ
                 + ∫(-mean_nn(v,n_Λ)⊙jump(∇(u)⋅n_Λ) - mean_nn(u,n_Λ)⊙jump(∇(v)⋅n_Λ))dΛ
                 + ∫(-( n_Γ ⋅ ∇∇(v)⋅ n_Γ )⊙∇(u)⋅n_Γ - ( n_Γ ⋅ ∇∇(u)⋅ n_Γ )⊙∇(v)⋅n_Γ)dΓ
                 + ∫((γ/h)⋅jump(∇(u)⋅n_Λ)⊙jump(∇(v)⋅n_Λ))dΛ + ∫((γ/h)⋅ ∇(u)⊙n_Γ⋅∇(v)⊙n_Γ )dΓ
                )

    # Define linear form
    # Notation: g_1 = ∇u_ex⋅n_Γ, g_2 = ∇Δu_ex⋅n_Γ
    g_1 = ∇u_ex⋅n_Γ
    g_2 = ∇Δu_ex⋅n_Γ
    l(v) = (∫( f*v ) * dΩ
            +  ∫(-(g_2⋅v))dΓ
            + ∫(g_1⊙(-(n_Γ⋅∇∇(v)⋅n_Γ) + (γ/h)*∇(v)⋅n_Γ)) * dΓ
            )

    # Define of ghost penalties
    if order == 2
        g(u,v) = h^(-2)*(∫( (γg0/h)*jump(u)*jump(v)) * dFg +
                        ∫( (γg1*h)*jump(n_Fg⋅∇(u))*jump(n_Fg⋅∇(v)) ) * dFg +
                         ∫( (γg2*h^3)*jump_nn(u,n_Fg)*jump_nn(v,n_Fg) ) * dFg)
    else
        println("Not supported order:", order)
    end

    A(u,v) = a(u,v) + g(u,v)


    # Assemble of system
    op = AffineFEOperator(a, l, U, V)
    uh = solve(op)
    A =  get_matrix(op)
    ndof = size(A)[1]
    h = 1/sqrt(ndof)
    cond_number = ( 1/sqrt(ndof) )*cond(A,Inf)

    e = u_ex - uh
    el2 = sqrt(sum( ∫(e*e)dΩ ))

    # TODO: Add α into ∫(e⊙e)*dΩ
    eh_energy = sqrt(sum( ∫(e⊙e)*dΩ + ∫( ∇∇(e)⊙∇∇(e) )*dΩ
                      + ( γ/h ) * ∫(jump(∇(e)⋅n_Λ) ⊙ jump(∇(e)⋅n_Λ))dΛ
                      + ( h/γ ) * ∫(mean_nn(e,n_Λ) ⊙ mean_nn(e,n_Λ))dΛ
                      + ( γ/h ) * ∫((∇(e)⋅n_Γ) ⊙ (∇(e)⋅n_Γ))dΓ
                      + ( h/γ ) * ∫(( n_Γ ⋅ ∇∇(e)⋅ n_Γ ) ⊙ ( n_Γ ⋅ ∇∇(e)⋅ n_Γ ))dΓ
                     ))

    eh1 = sqrt(sum( ∫( e⊙e + ∇(e)⊙∇(e) )*dΩ ))

    println("n=",n, " L2=", el2, " energy=", eh_energy  )
   
end

minimalistic_CutFEM (generic function with 1 method)

In [2]:

println("Homogenous Example")
u_ex(x) = (x[1]^2 + x[2]^2  - 1)^2*sin(2π*x[1])*cos(2π*x[2])
n = 2^2
minimalistic_CutFEM(order=2, u_ex=u_ex, n=n)

n = 2^6
minimalistic_CutFEM(order=2, u_ex=u_ex, n=n)


println("Nonhomogenous Example")
u_ex(x) = sin(2π*x[1])*cos(2π*x[2])
n = 2^2
minimalistic_CutFEM(order=2, u_ex=u_ex, n=n)
n = 2^6
minimalistic_CutFEM(order=2, u_ex=u_ex, n=n)

Homogenous Example
n=4
n=4 L2=0.3996521510998364 energy=37.119960059735604
n=64
n=64 L2=0.002071869175054371 energy=2.2728722369190555
Nonhomogenous Example
n=4
n=4 L2=1.9290859690933797 energy=70.72967556386526
n=64
n=64 L2=0.5305362307597881 energy=11.180141133612048


# Test of rhs for CutFEM

In [3]:
# α(x) = x[1]^2 + x[2]^2
α(x) = 1
function test_rhs_CutFEM(;order, n, u_ex, ∇u_ex, ∇Δu_ex, f)

    # Background model
    L = 1.11
    domain = (-L, L, -L, L)
    pmin = Point(-L, -L)
    pmax = Point(L, L)
    partition = (n,n)
    bgmodel = CartesianDiscreteModel(pmin, pmax, partition)

    # Implicit geometry
    R  = 1.0
    geo = disk(R)

    # Cut the background model
    cutgeo = cut(bgmodel, geo)
    cutgeo_facets = cut_facets(bgmodel,geo)

    # Set up interpolation mesh and function spaces
    Ω_act = Triangulation(cutgeo, ACTIVE)

    # Set up interpolation mesh and function spaces
    Ω_act = Triangulation(cutgeo, ACTIVE)

    # Construct function spaces
    V = TestFESpace(Ω_act, ReferenceFE(lagrangian, Float64, order), conformity=:H1)
    U = TrialFESpace(V)

    # Set up integration meshes, measures and normals
    Ω = Triangulation(cutgeo, PHYSICAL)
    Γ = EmbeddedBoundary(cutgeo)
    Λ = SkeletonTriangulation(cutgeo_facets)
    Fg = GhostSkeleton(cutgeo)

    # Set up integration measures
    degree = 2*order
    dΩ   = Measure(Ω, degree)
    dΓ   = Measure(Γ, degree)
    dΛ   = Measure(Λ, degree) # F_int
    dFg  = Measure(Fg, degree)

    # Set up normal vectors
    n_Γ = get_normal_vector(Γ)
    n_Λ = get_normal_vector(Λ)
    n_Fg = get_normal_vector(Fg)

    # Define weak form
    γ = 25*order*( order+1)

    # Ghost penalty parameter
    γg0 = γ
    γg1 = 0.1
    γg2 = 0.1

    # Mesh size
    h = L/n


    function mean_n(u,n)
        return 0.5*( u.plus⋅n.plus + u.minus⋅n.minus )
    end

    function mean_nn(u,n)
        return 0.5*( n.plus⋅ ∇∇(u).plus⋅ n.plus + n.minus ⋅ ∇∇(u).minus ⋅ n.minus )
    end

    function jump_nn(u,n)
        return ( n.plus⋅ ∇∇(u).plus⋅ n.plus - n.minus ⋅ ∇∇(u).minus ⋅ n.minus )
    end
    # Inner facets
    a(u,v) =( ∫( ∇∇(v)⊙∇∇(u) + α⋅(v⊙u) )dΩ
                 + ∫(-mean_nn(v,n_Λ)⊙jump(∇(u)⋅n_Λ) - mean_nn(u,n_Λ)⊙jump(∇(v)⋅n_Λ))dΛ
                 + ∫(-( n_Γ ⋅ ∇∇(v)⋅ n_Γ )⊙∇(u)⋅n_Γ - ( n_Γ ⋅ ∇∇(u)⋅ n_Γ )⊙∇(v)⋅n_Γ)dΓ
                 + ∫((γ/h)⋅jump(∇(u)⋅n_Λ)⊙jump(∇(v)⋅n_Λ))dΛ + ∫((γ/h)⋅ ∇(u)⊙n_Γ⋅∇(v)⊙n_Γ )dΓ
                )

    # Define linear form
    # Notation: g_1 = ∇u_ex⋅n_Γ, g_2 = ∇Δu_ex⋅n_Γ
    g_1 = ∇u_ex⋅n_Γ
    g_2 = ∇Δu_ex⋅n_Γ
    """
    l(v) = (
            ∫( f*v ) * dΩ
            +  ∫(-(g_2⋅v))dΓ
            + ∫(g_1⊙(-(n_Γ⋅∇∇(v)⋅n_Γ) + (γ/h)*∇(v)⋅n_Γ)) * dΓ
            )
    """
    #l(v) = (∫( f*v ) * dΩ)
   # l(v) =  (∫(0*v)* dΩ +  ∫(g_1⊙(-(n_Γ⋅∇∇(v)⋅n_Γ) + (γ/h)*∇(v)⋅n_Γ)) * dΓ)
    l(v) =  (∫(0*v)* dΩ )+  ∫(g_1⊙(-(n_Γ⋅∇∇(v)⋅n_Γ) + (γ/h)*∇(v)⋅n_Γ)) * dΓ


    # Define of ghost penalties
    if order == 2
        g(u,v) = h^(-2)*(∫( (γg0/h)*jump(u)*jump(v)) * dFg +
                        ∫( (γg1*h)*jump(n_Fg⋅∇(u))*jump(n_Fg⋅∇(v)) ) * dFg +
                         ∫( (γg2*h^3)*jump_nn(u,n_Fg)*jump_nn(v,n_Fg) ) * dFg)
    else
        println("Not supported order:", order)
    end

    A(u,v) = a(u,v) + g(u,v)


    # Assemble of system
    op = AffineFEOperator(a, l, U, V)
    uh = solve(op)
    A =  get_matrix(op)
    ndof = size(A)[1]
    cond_number = ( 1/sqrt(ndof) )*cond(A,Inf)
    
    b = get_vector(op)
    println("b = ", b, "n= ",n)
    
    """
    e = u_ex - uh
    el2 = sqrt(sum( ∫(e*e)dΩ ))

    # TODO: Add α into ∫(e⊙e)*dΩ
    eh_energy = sqrt(sum( ∫(e⊙e)*dΩ + ∫( ∇∇(e)⊙∇∇(e) )*dΩ
                      + ( γ/h ) * ∫(jump(∇(e)⋅n_Λ) ⊙ jump(∇(e)⋅n_Λ))dΛ
                      + ( h/γ ) * ∫(mean_nn(e,n_Λ) ⊙ mean_nn(e,n_Λ))dΛ
                      + ( γ/h ) * ∫((∇(e)⋅n_Γ) ⊙ (∇(e)⋅n_Γ))dΓ
                      + ( h/γ ) * ∫(( n_Γ ⋅ ∇∇(e)⋅ n_Γ ) ⊙ ( n_Γ ⋅ ∇∇(e)⋅ n_Γ ))dΓ
                     ))

    eh1 = sqrt(sum( ∫( e⊙e + ∇(e)⊙∇(e) )*dΩ ))

    println("n=",n, " L2=", el2, " energy=", eh_energy  )
    """
    0
end

test_rhs_CutFEM (generic function with 1 method)

In [4]:
r(x) = exp(-x[1]^2)
r_x(x) = -2*x[1]*exp(-x[1]^2)
r_y(x) = 0

u_ex(x) = (x[1]^2 + x[2]^2  - 1)^2*r(x)
∇Δu_ex(x) = ∇(Δ(u_ex))(x)
f(x) = Δ(Δ(u_ex))(x)+ α(x)⋅u_ex(x)

∇u_ex(x) = VectorValue(4*x[1]*(x[1]^2 + x[2]^2  - 1)*r(x) + (x[1]^2 + x[2]^2  - 1)^2*r_x(x),
                    4*x[2]*(x[1]^2 + x[2]^2  - 1)*r(x) + (x[1]^2 + x[2]^2  - 1)^2*r_y(x))

n = 2^2
test_rhs_CutFEM(order=2, n=n, u_ex=u_ex, ∇u_ex=∇u_ex, ∇Δu_ex=∇Δu_ex, f=f )

b = [-2.7077863048041353, -29.984145641977467, -90.60123479081156, -29.97169454179222, -2.7078745250790925, -25.078370754981734, 161.9283140970898, -14.700646379084976, 161.94105375928046, -25.07818001060879, -36.075947447187616, -9.978113547388787, 0.0, -9.978113547388853, -36.0759474471877, -25.07818001060878, 161.94105375928044, -14.700646379085061, 161.92831409708973, -25.078370754981783, -2.7078745250790943, -29.971694541792168, -90.60123479081176, -29.984145641977506, -2.707786304804145, 23.55005327136302, -34.78806338684918, 25.170018273021213, -64.19419891589055, -235.89455771216055, 120.69337816606371, 260.5127496795734, -235.91950942429665, 120.66738377032395, -64.21593735305001, 23.550566882768358, -34.78942940973752, 25.170839394655758, 116.38480266841574, -121.92515149701688, 67.33182593566777, 0.0, 0.0, 0.0, 67.32907595510368, 116.38480266841577, -121.92512160055321, -34.78942940973748, -121.92512160055323, 67.32907595510366, 120.66738377032391, 0.0, 120.69337816606367, 6

0

In [5]:
∇u_ex(x) = ∇(u_ex)(x)
n = 2^2
test_rhs_CutFEM(order=2, n=n, u_ex=u_ex, ∇u_ex=∇u_ex, ∇Δu_ex=∇Δu_ex, f=f )


b = [-2.7077863048041353, -29.984145641977467, -90.60123479081156, -29.97169454179222, -2.7078745250790925, -25.078370754981734, 161.9283140970898, -14.700646379084976, 161.94105375928046, -25.07818001060879, -36.075947447187616, -9.978113547388787, 0.0, -9.978113547388853, -36.0759474471877, -25.07818001060878, 161.94105375928044, -14.700646379085061, 161.92831409708973, -25.078370754981783, -2.7078745250790943, -29.971694541792168, -90.60123479081176, -29.984145641977506, -2.707786304804145, 23.55005327136302, -34.78806338684918, 25.170018273021213, -64.19419891589055, -235.89455771216055, 120.69337816606371, 260.5127496795734, -235.91950942429665, 120.66738377032395, -64.21593735305001, 23.550566882768358, -34.78942940973752, 25.170839394655758, 116.38480266841574, -121.92515149701688, 67.33182593566777, 0.0, 0.0, 0.0, 67.32907595510368, 116.38480266841577, -121.92512160055321, -34.78942940973748, -121.92512160055323, 67.32907595510366, 120.66738377032391, 0.0, 120.69337816606367, 6

0

# Minimal CIP Nitsche formulation

In [6]:
function minimalistic_CIP(;order, n,  u_ex)
    ∇u_ex(x) = ∇(u_ex)(x)
    ∇Δu_ex(x) = ∇(Δ(u_ex))(x)
    f(x) = Δ(Δ(u_ex))(x)+ α(x)⋅u_ex(x)
    
    refinement = Integer(log2(n)) - 1
    if ( refinement < 1 || refinement > 6 )
        println("Refinement out of range.")
    end
    mesh_file = "circle/circle_"*string(refinement)*".msh"
    model = GmshDiscreteModel(mesh_file)
    h = 2/n
        
    γ = 1.5*order*( order+1)

    # Spaces
    V = TestFESpace(model, ReferenceFE(lagrangian,Float64, order), conformity=:H1)
    U = TrialFESpace(V)
    Ω = Triangulation(model)
    Λ = SkeletonTriangulation(model)
    Γ = BoundaryTriangulation(model)

    degree = 2*order
    dΩ = Measure(Ω,degree)
    dΓ = Measure(Γ,degree)
    dΛ = Measure(Λ,degree)

    n_Λ = get_normal_vector(Λ)
    n_Γ = get_normal_vector(Γ)

    function mean_nn(u,n)
        return 0.5*( n.plus⋅ ∇∇(u).plus⋅ n.plus + n.minus ⋅ ∇∇(u).minus ⋅ n.minus )
    end

    # Inner facets
    a(u,v) =( ∫( ∇∇(v)⊙∇∇(u) + α⋅(v⊙u) )dΩ
            + ∫(-mean_nn(v,n_Λ)⊙jump(∇(u)⋅n_Λ) - mean_nn(u,n_Λ)⊙jump(∇(v)⋅n_Λ))dΛ
            + ∫(-( n_Γ ⋅ ∇∇(v)⋅ n_Γ )⊙∇(u)⋅n_Γ - ( n_Γ ⋅ ∇∇(u)⋅ n_Γ )⊙∇(v)⋅n_Γ)dΓ
            + ∫((γ/h)⋅jump(∇(u)⋅n_Λ)⊙jump(∇(v)⋅n_Λ))dΛ + ∫((γ/h)⋅ ∇(u)⊙n_Γ⋅∇(v)⊙n_Γ )dΓ
                )

    g_1 = ∇u_ex⋅n_Γ
    g_2 = ∇Δu_ex⋅n_Γ

    l(v) = (∫( f*v ) * dΩ
            +  ∫(-(g_2⋅v))dΓ
            + ∫(g_1⊙(-(n_Γ⋅∇∇(v)⋅n_Γ) + (γ/h)*∇(v)⋅n_Γ)) * dΓ
            )

    op = AffineFEOperator(a, l, U, V)
    uh = solve(op)
    A =  get_matrix(op)
    ndof = size(A)[1]
    h = 1/sqrt(ndof)

    e = u_ex - uh
    el2 = sqrt(sum( ∫(e*e)dΩ ))

    # TODO: Add α into ∫(e⊙e)*dΩ
    eh_energy = sqrt(sum( ∫(e⊙e)*dΩ + ∫( ∇∇(e)⊙∇∇(e) )*dΩ
                      + ( γ/h ) * ∫(jump(∇(e)⋅n_Λ) ⊙ jump(∇(e)⋅n_Λ))dΛ
                      + ( h/γ ) * ∫(mean_nn(e,n_Λ) ⊙ mean_nn(e,n_Λ))dΛ
                      + ( γ/h ) * ∫((∇(e)⋅n_Γ) ⊙ (∇(e)⋅n_Γ))dΓ
                      + ( h/γ ) * ∫(( n_Γ ⋅ ∇∇(e)⋅ n_Γ ) ⊙ ( n_Γ ⋅ ∇∇(e)⋅ n_Γ ))dΓ
                     ))

    eh1 = sqrt(sum( ∫( e⊙e + ∇(e)⊙∇(e) )*dΩ ))

    u_inter = interpolate(u_ex, V)
    println("n=",n, " L2=", el2, " energy=", eh_energy  )
end 

minimalistic_CIP (generic function with 1 method)

In [7]:

println("Homogenous Example")
u_ex(x) = (x[1]^2 + x[2]^2  - 1)^2*sin(2π*x[1])*cos(2π*x[2])
n = 2^2
minimalistic_CIP(order=2, u_ex=u_ex, n=n)

n = 2^6
minimalistic_CIP(order=2, u_ex=u_ex, n=n)


println("Nonhomogenous Example")
u_ex(x) = sin(2π*x[1])*cos(2π*x[2])
n = 2^2
minimalistic_CIP(order=2, u_ex=u_ex, n=n)
n = 2^6
minimalistic_CIP(order=2, u_ex=u_ex, n=n)

Homogenous Example
Info    : Reading 'circle/circle_1.msh'...
Info    : 3 entities
Info    : 60 nodes
Info    : 119 elements
Info    : Done reading 'circle/circle_1.msh'
n=4 L2=0.14939912669992605 energy=50.57828511184713
Info    : Reading 'circle/circle_5.msh'...
Info    : 3 entities
Info    : 12345 nodes
Info    : 24689 elements
Info    : Done reading 'circle/circle_5.msh'
n=64 L2=0.0008580481696148493 energy=3.1128810620454694
Nonhomogenous Example
Info    : Reading 'circle/circle_1.msh'...
Info    : 3 entities
Info    : 60 nodes
Info    : 119 elements
Info    : Done reading 'circle/circle_1.msh'
n=4 L2=0.4706145997694027 energy=86.46392286662402
Info    : Reading 'circle/circle_5.msh'...
Info    : 3 entities
Info    : 12345 nodes
Info    : 24689 elements
Info    : Done reading 'circle/circle_5.msh'
n=64 L2=0.5937849020159831 energy=12.301225317394795


 # Test of rhs for CIP Nitsche


In [8]:
function test_rhs_CIP(;order, n, u_ex, ∇u_ex, ∇Δu_ex, f)
   
    
    refinement = Integer(log2(n)) - 1
    if ( refinement < 1 || refinement > 6 )
        println("Refinement out of range.")
    end
    mesh_file = "circle/circle_"*string(refinement)*".msh"
    model = GmshDiscreteModel(mesh_file)
    h = 2/n
        
    γ = 1.5*order*( order+1)

    # Spaces
    V = TestFESpace(model, ReferenceFE(lagrangian,Float64, order), conformity=:H1)
    U = TrialFESpace(V)
    Ω = Triangulation(model)
    Λ = SkeletonTriangulation(model)
    Γ = BoundaryTriangulation(model)

    degree = 2*order
    dΩ = Measure(Ω,degree)
    dΓ = Measure(Γ,degree)
    dΛ = Measure(Λ,degree)

    n_Λ = get_normal_vector(Λ)
    n_Γ = get_normal_vector(Γ)

    function mean_nn(u,n)
        return 0.5*( n.plus⋅ ∇∇(u).plus⋅ n.plus + n.minus ⋅ ∇∇(u).minus ⋅ n.minus )
    end

    # Inner facets
    a(u,v) =( ∫( ∇∇(v)⊙∇∇(u) + α⋅(v⊙u) )dΩ
            + ∫(-mean_nn(v,n_Λ)⊙jump(∇(u)⋅n_Λ) - mean_nn(u,n_Λ)⊙jump(∇(v)⋅n_Λ))dΛ
            + ∫(-( n_Γ ⋅ ∇∇(v)⋅ n_Γ )⊙∇(u)⋅n_Γ - ( n_Γ ⋅ ∇∇(u)⋅ n_Γ )⊙∇(v)⋅n_Γ)dΓ
            + ∫((γ/h)⋅jump(∇(u)⋅n_Λ)⊙jump(∇(v)⋅n_Λ))dΛ + ∫((γ/h)⋅ ∇(u)⊙n_Γ⋅∇(v)⊙n_Γ )dΓ
                )

    g_1 = ∇u_ex⋅n_Γ
    g_2 = ∇Δu_ex⋅n_Γ
    
    """
    l(v) = (
            ∫( f*v ) * dΩ
            +  ∫(-(g_2⋅v))dΓ
            + ∫(g_1⊙(-(n_Γ⋅∇∇(v)⋅n_Γ) + (γ/h)*∇(v)⋅n_Γ)) * dΓ
            )
    """
    # l(v) = (∫( f*v ) * dΩ)
    # l(v) =  (∫(0*v)* dΩ +  ∫(g_1⊙(-(n_Γ⋅∇∇(v)⋅n_Γ) + (γ/h)*∇(v)⋅n_Γ)) * dΓ)
    l(v) =  (∫(0*v)* dΩ )+  ∫(g_1⊙(-(n_Γ⋅∇∇(v)⋅n_Γ) + (γ/h)*∇(v)⋅n_Γ)) * dΓ

    op = AffineFEOperator(a, l, U, V)
    uh = solve(op)
    A =  get_matrix(op)
    
    b = get_vector(op)
    println("b = ", b, "n= ",n)
    """

    e = u_ex - uh
    el2 = sqrt(sum( ∫(e*e)dΩ ))

    # TODO: Add α into ∫(e⊙e)*dΩ
    eh_energy = sqrt(sum( ∫(e⊙e)*dΩ + ∫( ∇∇(e)⊙∇∇(e) )*dΩ
                      + ( γ/h ) * ∫(jump(∇(e)⋅n_Λ) ⊙ jump(∇(e)⋅n_Λ))dΛ
                      + ( h/γ ) * ∫(mean_nn(e,n_Λ) ⊙ mean_nn(e,n_Λ))dΛ
                      + ( γ/h ) * ∫((∇(e)⋅n_Γ) ⊙ (∇(e)⋅n_Γ))dΓ
                      + ( h/γ ) * ∫(( n_Γ ⋅ ∇∇(e)⋅ n_Γ ) ⊙ ( n_Γ ⋅ ∇∇(e)⋅ n_Γ ))dΓ
                     ))

    eh1 = sqrt(sum( ∫( e⊙e + ∇(e)⊙∇(e) )*dΩ ))
      
    b = get_vector(op)
    println("b = ", b, "n= ",n)
    
    u_inter = interpolate(u_ex, V)
    println("n=",n, " L2=", el2, " energy=", eh_energy  )
    """
    0
end 

test_rhs_CIP (generic function with 1 method)

In [9]:
r(x) = exp(-x[1]^2)
r_x(x) = -2*x[1]*exp(-x[1]^2)
r_y(x) = 0

u_ex(x) = (x[1]^2 + x[2]^2  - 1)^2*r(x)
∇Δu_ex(x) = ∇(Δ(u_ex))(x)
f(x) = Δ(Δ(u_ex))(x)+ α(x)⋅u_ex(x)

∇u_ex(x) = VectorValue(4*x[1]*(x[1]^2 + x[2]^2  - 1)*r(x) + (x[1]^2 + x[2]^2  - 1)^2*r_x(x),
                    4*x[2]*(x[1]^2 + x[2]^2  - 1)*r(x) + (x[1]^2 + x[2]^2  - 1)^2*r_y(x))

n = 2^2
test_rhs_CIP(order=2, n=n, u_ex=u_ex, ∇u_ex=∇u_ex, ∇Δu_ex=∇Δu_ex, f=f )


Info    : Reading 'circle/circle_1.msh'...
Info    : 3 entities
Info    : 60 nodes
Info    : 119 elements
Info    : Done reading 'circle/circle_1.msh'
b = [-0.16924613442328035, -0.19042202553265522, -0.13334927836834246, -0.21864976294737082, -0.3561309834125693, -0.3803518211322136, -0.4072927064259645, -0.26471964071084214, -0.3355355553853989, -0.2519681012904279, -0.11577812435691127, -0.13831557760287158, -0.1761443149825231, -0.21158529503879542, -0.1960467853201123, -0.2790391406589514, -0.35641301853374735, -0.5038317209461302, -0.5030713139083154, -0.37629478138365124, -0.23641762169090075, -0.1637792950468446, -0.14435753173973664, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.06806432563509764, -0.10663681963805512, -0.17494331739845684, -0.07521220271121262, -0.18073296751099777, -0.10729303751795961, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.08067667610694712, -0.07763358045688803, -0.09227932523902192, -0.0997094479231608, -0.24254276284131887, -0.055250888818258896, -0.0338943584175101,

0

In [10]:
∇u_ex(x) = ∇(u_ex)(x)
n = 2^2
test_rhs_CIP(order=2, n=n, u_ex=u_ex, ∇u_ex=∇u_ex, ∇Δu_ex=∇Δu_ex, f=f )


Info    : Reading 'circle/circle_1.msh'...
Info    : 3 entities
Info    : 60 nodes
Info    : 119 elements
Info    : Done reading 'circle/circle_1.msh'
b = [-0.16924613442328035, -0.19042202553265522, -0.13334927836834246, -0.21864976294737082, -0.3561309834125693, -0.3803518211322136, -0.4072927064259645, -0.26471964071084214, -0.3355355553853989, -0.2519681012904279, -0.11577812435691127, -0.13831557760287158, -0.1761443149825231, -0.21158529503879542, -0.1960467853201123, -0.2790391406589514, -0.35641301853374735, -0.5038317209461302, -0.5030713139083154, -0.37629478138365124, -0.23641762169090075, -0.1637792950468446, -0.14435753173973664, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.06806432563509764, -0.10663681963805512, -0.17494331739845684, -0.07521220271121262, -0.18073296751099777, -0.10729303751795961, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.08067667610694712, -0.07763358045688803, -0.09227932523902192, -0.0997094479231608, -0.24254276284131887, -0.055250888818258896, -0.0338943584175101,

0